## YOLO approach for RSNA Brain Tumor
* The YOLO system includes a function to train a classifier to identify the object.
 * https://github.com/ultralytics/yolov5
* To do it, need to manually create annotation data to train the classifier. 
* Annotation work is conducted manually using png-converted dataset and an annotation tool, LabelMe.
 * https://www.kaggle.com/jonathanbesomi/rsna-miccai-png
 * https://github.com/wkentaro/labelme
* This notebook shows the process of converting json annotation data to DataFrame.
* If YOLO is used in this competition, it is required that the trained classifier never produce false positive judgement, because there are dozens of images per patient.

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import json
import codecs
from pandas.io.json import json_normalize

In [ ]:
files=[]
for dirname, _, filenames in os.walk('../input/rsna-miccai-brain-annotation/rsna_biccai_brain_0916'):
    for filename in filenames:
        files+=[os.path.join(dirname, filename)]

In [ ]:
imgpath=[]
for item in files:
    if item[-4:]=='.png':
          imgpath+=[item]
print(imgpath[0:5])
print(len(imgpath))

In [ ]:
jsonpath=[]
for item in files:
    if item[-4:]=='json':
          jsonpath+=[item]
print(jsonpath[0:5])
print(len(jsonpath))

In [ ]:
box=[]
with open('/kaggle/input/rsna-miccai-brain-annotation/rsna_biccai_brain_0916/0000/Image-88.json') as f:
    d = json.load(f)
    print(d)

In [ ]:
jsonpath[0]

In [ ]:
BOX=[]
for path in jsonpath:
    with open(path) as f:
        idn=path.split('/')[-2]
        d = json.load(f)
        labels=d['shapes']
        height=d['imageHeight']
        width=d['imageWidth']
        file=path[0:-5].split('/')[-1]
        for item in labels:
            BOX+=[[idn,file,item['label'],item['points'][0][0],item['points'][0][1],item['points'][1][0],item['points'][1][1],height,width]]
print(BOX[0:5])    

In [ ]:
data0=pd.DataFrame(BOX)
data0.columns=['idn','file','name','ymin','xmin','ymax','xmax','height','width']
data0[0:5]

In [ ]:
data0['label']=data0['name'].map({'non_tumor':0,'tumor':1})
data0['Ycent']=(data0['ymin']+data0['ymax'])/(2*data0['height'])
data0['Xcent']=(data0['xmin']+data0['xmax'])/(2*data0['width'])
data0['boxH']=(data0['ymax']-data0['ymin'])/data0['height']
data0['boxW']=(data0['xmax']-data0['xmin'])/data0['width']
data0[0:5]

In [ ]:
data0.to_csv('brain_tumor_annotation.csv',index=False)

In [ ]:
data0[data0['file']=='Image-88'][data0['idn']=='0000']

In [ ]:
items=data0[data0['file']=='Image-88'][data0['idn']=='0000'].iloc[:,0:6].reset_index(drop=True)
print(len(items))
print(items.loc[0])

In [ ]:
imgpath[322]

In [ ]:
for i,item in enumerate(imgpath):
    if item[0:-4].split('/')[-1]=='Image-88' and item[0:-4].split('/')[-2]=='0000':
          print(i)

## cv2.rectangle(image,(ymin,xmin),(ymax,xmax),(0,255,0),2)
* https://www.geeksforgeeks.org/python-opencv-cv2-rectangle-method/

In [ ]:
def draw_bbox2(path):

    items=data0[data0['file']==path[0:-4].split('/')[-1]][data0['idn']==path[0:-4].split('/')[-2]].iloc[:,0:13].reset_index(drop=True)
    image=cv2.imread(path)
    idn=path[0:-4].split('/')[-2]
    file=path[0:-4].split('/')[-1]    
    print(idn,file)
    if len(items)>0:
        for i in range(len(items)):
            label=items.loc[i,'label'].astype(int)
            xmin=items.loc[i,'xmin'].astype(int)
            ymin=items.loc[i,'ymin'].astype(int)
            xmax=items.loc[i,'xmax'].astype(int)
            ymax=items.loc[i,'ymax'].astype(int)
            H=items.loc[i,'height'].astype(int)
            W=items.loc[i,'width'].astype(int)            
            print(label,ymin,xmin,ymax,xmax)
            if label==0:
                cv2.rectangle(image,(ymin,xmin),(ymax,xmax),(0,255,0),2)
            elif label==1:
                cv2.rectangle(image,(ymin,xmin),(ymax,xmax),(0,0,255),2)         
            
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()       

In [ ]:
draw_bbox2(imgpath[322])

In [ ]:
for item in imgpath[0:6]:
    draw_bbox2(item)